In [2]:
import nltk
from nltk.tag.stanford import StanfordNERTagger
import os
import re
import pandas as pd
from tqdm import tqdm
from copy import copy

java_path = r"C:\Program Files\AdoptOpenJDK\jdk-11.0.7.10-hotspot\bin\java.exe"
PATH_TO_JAR= r"C:\Users\effbl\stanza_corenlp\stanford-ner-2020-11-17\stanford-ner.jar"
PATH_TO_MODEL = r"C:\Users\effbl\stanza_corenlp\stanford-ner-2020-11-17\classifiers\english.muc.7class.distsim.crf.ser.gz"

os.environ['JAVAHOME'] = java_path
tagger = StanfordNERTagger(model_filename=PATH_TO_MODEL,path_to_jar=PATH_TO_JAR, encoding='utf-8')

In [4]:
text = 'The company, which made its last recall on Oct. 2 after several cars had lost power and stopped running, is recalling close to 4,600 vehicles due to a faulty component.'
words = nltk.word_tokenize(text)
tagged = tagger.tag(words)

In [5]:
tagged

[('The', 'O'),
 ('company', 'O'),
 (',', 'O'),
 ('which', 'O'),
 ('made', 'O'),
 ('its', 'O'),
 ('last', 'O'),
 ('recall', 'O'),
 ('on', 'O'),
 ('Oct.', 'DATE'),
 ('2', 'DATE'),
 ('after', 'O'),
 ('several', 'O'),
 ('cars', 'O'),
 ('had', 'O'),
 ('lost', 'O'),
 ('power', 'O'),
 ('and', 'O'),
 ('stopped', 'O'),
 ('running', 'O'),
 (',', 'O'),
 ('is', 'O'),
 ('recalling', 'O'),
 ('close', 'O'),
 ('to', 'O'),
 ('4,600', 'O'),
 ('vehicles', 'O'),
 ('due', 'O'),
 ('to', 'O'),
 ('a', 'O'),
 ('faulty', 'O'),
 ('component', 'O'),
 ('.', 'O')]